In [73]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
from met_explore.peak_selection import PeakSelector
from met_explore.compound_selection import CompoundSelector
from met_explore.preprocessing import *
import pandas as pd


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


### Populate Samples: 
> populate_samples(sample_csv)

This simply takes the sample_csv and populates the sample table with name, group, life-stage, tissue and mutant.

### Preprocess the compounds: 
> pre_peak_df = peak_select.pre_process_compounds()

This part of the pipeline is decicated to preparing the compounds so that we have a sensible compound DB. A file was stored at each different stage of processing to make it easy to go back and change methods when testing.
This uses the 1160_peak_cmpd_export.json file from PiMP and the CHEBI.owl file. It does not access the DB at this stage. This just adds identifiers and makes sure each unique compound is sensible.


In [3]:
peak_select = PeakSelector('/Users/Karen/FlyOmics/data/1160_peak_cmpd_export.json', '/Users/Karen/FlyOmics/data/1160_peak_int_export.json')


## The original peak DF from PiMP

In [6]:
original_peak_df = pd.read_json('/Users/Karen/FlyOmics/data/1160_peak_cmpd_export.json')
original_peak_df.head()

,pid,sec_id,mass,rt,polarity,cmpd_id,formula,adduct,identified,rc_id,compound,db,identifier,frank_annot,inchikey
0,2690228,1,116.070602,658.553657,positive,1,C5H9NO2,M+H,False,13261729,Pterolactam,hmdb,HMDB34208,"{'frank_cmpd_name': 'Proline', 'inchikey': Non...",VULIHENHKGDFAB-UHFFFAOYSA-N
1,2690819,592,116.070628,676.945093,positive,1,C5H9NO2,M+H,False,13266081,Pterolactam,hmdb,HMDB34208,None,VULIHENHKGDFAB-UHFFFAOYSA-N
2,2691680,1453,157.097252,658.383356,positive,1,C5H9NO2,M+ACN+H,False,13270709,Pterolactam,hmdb,HMDB34208,None,VULIHENHKGDFAB-UHFFFAOYSA-N
3,2692994,2767,116.070624,752.859086,positive,1,C5H9NO2,M+H,False,13281590,Pterolactam,hmdb,HMDB34208,None,VULIHENHKGDFAB-UHFFFAOYSA-N
4,2692995,2768,116.070625,765.635769,positive,1,C5H9NO2,M+H,False,13281615,Pterolactam,hmdb,HMDB34208,None,VULIHENHKGDFAB-UHFFFAOYSA-N


## Add neutral masses
#### The original_peak_df above DF with neutral masses and several updated inchikeys. There are stds_db compounds where the compound formulas don't match the inchi-keys so just updating them.

In [10]:
nm_inchi_df = peak_select.prepare_df(original_peak_df)
nm_inchi_df.head()

Preparing the DF
The file has been found: current_prepared_df 


,pid,sec_id,mass,rt,polarity,cmpd_id,formula,adduct,identified,rc_id,compound,db,identifier,frank_annot,inchikey,neutral_mass
0,2963849,1,116.070610,658.557235,positive,1,C5H9NO2,M+H,False,15043572,Pterolactam,hmdb,HMDB34208,"{'frank_cmpd_name': 'L-PROLINE', 'inchikey': N...",VULIHENHKGDFAB-UHFFFAOYSA-N,115.063334
1,2966393,2545,116.070630,432.801172,positive,1,C5H9NO2,M+H,False,15065769,Pterolactam,hmdb,HMDB34208,None,VULIHENHKGDFAB-UHFFFAOYSA-N,115.063354
2,2966448,2600,157.097251,423.968558,positive,1,C5H9NO2,M+ACN+H,False,15066454,Pterolactam,hmdb,HMDB34208,None,VULIHENHKGDFAB-UHFFFAOYSA-N,115.063425
3,2968039,4191,114.056056,659.098064,negative,1,C5H9NO2,M-H,False,15074909,Pterolactam,hmdb,HMDB34208,None,VULIHENHKGDFAB-UHFFFAOYSA-N,115.063332
4,2963849,1,116.070610,658.557235,positive,2,C5H9NO2,M+H,True,15043573,L-Proline,hmdb,HMDB00162,"{'frank_cmpd_name': 'L-PROLINE', 'inchikey': N...",ONIBWKKTOPOVIA-BYPYZUCNSA-N,115.063334


### Pass this DF to the PreprocessCompounds class
#### The details of the preprocessing of the compounds is given below but can be run from the single command:
> peak_chebi_df = pre_processor.get_preprocessed_cmpds()
#### This gives a dataframe where the cmpd_id should represent a single compound.

In [60]:
pre_processor = PreprocessCompounds(nm_inchi_df)
pre_peak_df = pre_processor.get_preprocessed_cmpds()


Getting the chebi_ontology df
The file chebi_peak_df_current has been found: 
Adding the chebi_ids
The file chebi_peak_df_current has been found: 
Making sure each Chebi_ID has the same cmpd_id
The file chebi_peak_df_cmpd_match_current has been found: 
Checking if chebi, inchi and formulas all have unique cmpd_ids
The file chebi_unique_cmpd_ids_current has been found: 
The file current_chebi_peak_df has been found: 


### Constructing a DF where each compound for a unique peak is given a single row (collecting Identifiers and DBs")
> construct_peak_df = peak_select.construct_all_peak_df(pre_peak_df)


In [64]:
construct_peak_df = peak_select.construct_all_peak_df(pre_peak_df)
display (construct_peak_df.head())

Constructing a DF where each compound for a unique peak is given a single row (collecting Identifiers and DBs
The file peak_prepared_df has been found: 


There are 3454 unique peaks out of 34799 rows added to the all_peak_df


,pid,sec_id,mass,rt,polarity,cmpd_id,formula,adduct,identified,rc_id,compound,db,identifier,frank_annot,inchikey,neutral_mass,chebi_id,chebi_name,cas_code,smiles
0,2963849,1,116.07061,658.557235,positive,1,C5H9NO2,M+H,False,15043572,[Pterolactam],[hmdb],[HMDB34208],"{'frank_cmpd_name': 'L-PROLINE', 'inchikey': N...",VULIHENHKGDFAB-UHFFFAOYSA-N,115.063334,None,None,NaN,NaN
4,2963849,1,116.07061,658.557235,positive,2,C5H9NO2,M+H,True,15043573,"[L-Proline, L-Proline, L-Proline]","[hmdb, kegg, stds_db]","[HMDB00162, C00148, Std_1_Dec18_19]","{'frank_cmpd_name': 'L-PROLINE', 'inchikey': N...",ONIBWKKTOPOVIA-BYPYZUCNSA-N,115.063334,17203,L-proline,147-85-3,OC(=O)[C@@H]1CCCN1
14,2963849,1,116.07061,658.557235,positive,3,C3H6O2,M+ACN+H,False,15043576,"[Propionic acid, Propanoate, Propionic acid]","[hmdb, kegg, lipidmaps]","[HMDB00237, C00163, LMFA01010003]","{'frank_cmpd_name': 'L-PROLINE', 'inchikey': N...",XBDQKXXYIPTUBI-UHFFFAOYSA-N,74.036784,30768,propionic acid,79-09-4,CCC(O)=O
20,2963849,1,116.07061,658.557235,positive,4,C5H9NO2,M+H,False,15043579,[4-Amino-2-methylenebutanoic acid],[hmdb],[HMDB30409],"{'frank_cmpd_name': 'L-PROLINE', 'inchikey': N...",FTWHFXMUJQRNBK-UHFFFAOYSA-N,115.063334,None,None,NaN,NaN
24,2963849,1,116.07061,658.557235,positive,5,C5H9NO2,M+H,False,15043580,"[Acetamidopropanal, 3-Acetamidopropanal]","[hmdb, kegg]","[HMDB12880, C18170]","{'frank_cmpd_name': 'L-PROLINE', 'inchikey': N...",ARJPPNFIEQKVBB-UHFFFAOYSA-N,115.063334,30322,3-acetamidopropanal,NaN,[H]C(=O)CCNC(C)=O


### Remove duplicate peaks based on mz and RT tolerance levels 
> peak_df = peak_select.remove_duplicates(construct_peak_df)

In [66]:
peak_df = peak_select.remove_duplicates(construct_peak_df)

Removing duplicate peaks based on mz and RT tolerance levels
The file dup_removed_peak_df has been found: 
Returning the peak_df without duplicate peaks


### Use the above DF to populate the peaks, compounds and annotations.

## Inside the preprocessor class

### Within this class we construct a chebi ontology df from the chebi.owl file.

In [16]:
pre_processor.construct_chebi_ontology_df()

Getting the chebi_ontology df
The file chebi_peak_df_current has been found: 


,chebi_id,chebi_name,chebi_formula,chebi_mass,chebi_mmass,chebi_charge,hmdb_id,kegg_id,lmap_id,cas_code,smiles,inchikey
0,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10,(+)-Atherospermoline,C36H38N2O6,594.698,594.27299,0,NaN,C11141,NaN,21008-67-3,COc1cc2CCN(C)[C@H]3Cc4ccc(Oc5cc(C[C@@H]6N(C)CC...,XGEAUXVPBXUBKN-NSOVKSMOSA-N
2,100,(-)-medicarpin,C16H14O4,270.27996,270.08921,0,NaN,C10503,NaN,32383-76-9,[H][C@@]12COc3cc(O)ccc3[C@]1([H])Oc1cc(OC)ccc21,NSRJSISNDPOJOP-BBRMVZONSA-N
3,1000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10000,Vismione D,C25H30O5,410.504,410.20932,0,NaN,C09977,NaN,87605-72-9,CC(C)=CCC\C(C)=C\COc1cc(O)c2c(O)c3C(=O)CC(C)(O...,KZPCPZBBGCTGCN-LZYBPNLTSA-N
...,...,...,...,...,...,...,...,...,...,...,...,...
132077,99995,"2-[(2S,4aS,12aS)-5-methyl-6-oxo-8-[(1-oxo-2-ph...",C27H33N3O6,495.568,495.23694,0,NaN,NaN,NaN,NaN,CN1[C@H]2CC[C@H](O[C@@H]2COC3=C(C1=O)C=C(C=C3)...,CUJGJHGFROPBKV-ODGPQVTHSA-N
132078,99996,"N-[(1S,3S,4aR,9aS)-3-[2-[(2,5-difluorophenyl)m...",C29H26F2N2O7,552.524,552.17081,0,NaN,NaN,NaN,NaN,C1[C@H](O[C@H]([C@@H]2[C@H]1C3=C(O2)C=CC(=C3)N...,JRAPIMFHFBSUKE-HCJMUSFBSA-N
132079,99997,"N-[(2S,4aS,12aS)-2-[2-(cyclohexylmethylamino)-...",C30H36FN3O5,537.623,537.26390,0,NaN,NaN,NaN,NaN,CN1[C@H]2CC[C@H](O[C@@H]2COC3=C(C1=O)C=C(C=C3)...,BSKZHQCAOYVBFU-SCTDOJESSA-N
132080,99998,"N-[[(3S,9S,10R)-16-(dimethylamino)-12-[(2S)-1-...",C28H43N3O6S2,581.791,581.25933,0,NaN,NaN,NaN,NaN,C[C@H]1CCCCO[C@@H]([C@@H](CN(C(=O)C2=C(O1)C=CC...,ANVCKVJMNHXGBG-FYBXHKAZSA-N


### Add Chebi Ids to the DF
> pre_processor.add_chebi_ids()
#### For each row in the nm_inchi_df get the chebi_id from the DB identifiers and if these are not in the DB get it from the inchikey.   Chebi names, smile and cas-codes are also added.


In [43]:
pre_processor.add_chebi_ids()
peak_df1 = pd.read_pickle("./data/chebi_peak_df_current.pkl")
peak_df1.head()

Adding the chebi_ids
The file chebi_peak_df_current has been found: 


,pid,sec_id,mass,rt,polarity,cmpd_id,formula,adduct,identified,rc_id,compound,db,identifier,frank_annot,inchikey,neutral_mass,chebi_id,chebi_name,cas_code,smiles
0,2963849,1,116.070610,658.557235,positive,1,C5H9NO2,M+H,False,15043572,Pterolactam,hmdb,HMDB34208,"{'frank_cmpd_name': 'L-PROLINE', 'inchikey': N...",VULIHENHKGDFAB-UHFFFAOYSA-N,115.063334,None,None,NaN,NaN
1,2966393,2545,116.070630,432.801172,positive,1,C5H9NO2,M+H,False,15065769,Pterolactam,hmdb,HMDB34208,None,VULIHENHKGDFAB-UHFFFAOYSA-N,115.063354,None,None,NaN,NaN
2,2966448,2600,157.097251,423.968558,positive,1,C5H9NO2,M+ACN+H,False,15066454,Pterolactam,hmdb,HMDB34208,None,VULIHENHKGDFAB-UHFFFAOYSA-N,115.063425,None,None,NaN,NaN
3,2968039,4191,114.056056,659.098064,negative,1,C5H9NO2,M-H,False,15074909,Pterolactam,hmdb,HMDB34208,None,VULIHENHKGDFAB-UHFFFAOYSA-N,115.063332,None,None,NaN,NaN
4,2963849,1,116.070610,658.557235,positive,2,C5H9NO2,M+H,True,15043573,L-Proline,hmdb,HMDB00162,"{'frank_cmpd_name': 'L-PROLINE', 'inchikey': N...",ONIBWKKTOPOVIA-BYPYZUCNSA-N,115.063334,17203,L-proline,147-85-3,OC(=O)[C@@H]1CCCN1


### Give all rows with the same chebi_id the same cmpd ID in the peak_df
> give_each_chebi_same_id()

In [55]:
pre_processor.give_each_chebi_same_id()
peak_df2 = pd.read_pickle("./data/chebi_peak_df_cmpd_match_current.pkl")

Making sure each Chebi_ID has the same cmpd_id
The file chebi_peak_df_cmpd_match_current has been found: 


### Make sure all unique cmpds have unique chebi ids. If there are duplicate compounds with  the same ID and different chebi_ids, inchi_keys or formulas - make a new cmpd.
>give_chebi_inchi_unique_id()

In [56]:
pre_processor.give_chebi_inchi_unique_id()
peak_df3 = pd.read_pickle("./data/chebi_unique_cmpd_ids_current.pkl")

Checking if chebi, inchi and formulas all have unique cmpd_ids
The file chebi_unique_cmpd_ids_current has been found: 


### A method to collect any std_cmpds with no chebi and match them to other cmpds with the same Inchikeys
> change_std_cmpds_no_chebi ()

In [58]:
pre_processor.change_std_cmpds_no_chebi()
peak_df4 = pd.read_pickle("./data/current_chebi_peak_df.pkl")

### Populate the peaks, compounds and annotations for the filtered peaks 
> populate_peaks_cmpds_annots(peak_df))



### Construct the intensity DF

> int_df, ids_dict = peak_select.construct_int_df(peak_df)

This just filters down the intensity file given back by PiMP to just contain the peaks that have annotations associated with them. This file contains 9369 out of the original 27574 peaks.


In [ ]:
int_df, ids_dict = peak_select.construct_int_df(peak_df)
display (int_df.head())

### Use this to populate the peak/samples
>  populate_peaksamples(int_df, ids_dict)


### Select a DF of peaks based on most likely annotations
> selected_df, unique_sec_ids = peak_select.get_selected_df(peak_df)

This selects peaks with annotations that have M+H/M-H/M and that have been identifed or have a FrAnK annotation.


In [ ]:
selected_df, unique_sec_ids = peak_select.get_selected_df(peak_df)
display(selected_df.head())
print ("This has", len(selected_df['sec_id'].unique()), "sec_ids out of", selected_df.shape[0], "peak rows")


### Construct the High confidence peak DF
> high_conf_peak_df = peak_select.construct_high_confidence_peak_df(selected_df, unique_sec_ids)

This method chooses the the best annotation for each of the peaks based on a number of factors including wether or not the compound was identified as being this peak. At this stage for many of these peaks the smae compound has been chosen. 


In [ ]:
high_conf_peak_df = peak_select.construct_high_confidence_peak_df(selected_df, unique_sec_ids)
display(high_conf_peak_df.head())

In [ ]:
high_conf_peak_df[high_conf_peak_df['cmpd_id']==2]

### Choose a single peak for a single compound so that we can highlight this as a preferred annotation.

The first thing we do is to construct a DF with all the intensity values for all of the peaks in the high_confidence Dataframe. Here the intensities were taken from the popukated DB but the peak and compound details were taken from the DF above.
> compound_select = CompoundSelector()

> hc_int_df = compound_select.construct_hc_int_df(high_conf_peak_df)


In [ ]:
compound_select = CompoundSelector()
hc_int_df = compound_select.construct_hc_int_df(high_conf_peak_df)
display(hc_int_df[hc_int_df['cmpd_id']==9162])

In [ ]:
thy_df = hc_int_df[hc_int_df['Metabolite']=="thymine"]
test_df = thy_df[['FM_TRA_L1.mzXML','FM_TRA_L2.mzXML', 'FM_TRA_L3.mzXML', 'FM_TRA_L4.mzXML']]

wtf_df = compound_select.get_group_df(thy_df)


In [ ]:
display(wtf_df[['Trachea_l']])

### Obtain a DF that has a single peak for a single high confidence compound. 

This is done simply on choosing the peak that has the highest group (tissue) intensity value). This leaves us with 132 compounds that have been assigned to a single peak.

> single_cmpds_df = compound_select.get_single_cmpd_df(hc_int_df)


In [ ]:
single_cmpds_df = compound_select.get_single_cmpd_df(hc_int_df)
display(single_cmpds_df.head())
single_cmpds_df.shape

In [ ]:
single_cmpds_df[single_cmpds_df['Metabolite']=='thymine']